In [3]:
import PyPDF2
#following: https://www.codespeedy.com/find-the-page-number-of-a-text-from-a-pdf-file-in-python/
import re
import pandas as pd
#also: pip install openpyxl

In [6]:

file="Maintext.pdf"
folder="XXXXXXX/"
file2="listofwords.xlsx"

In [ ]:
listdf=pd.read_excel(folder+file2, header=None)
listdf

In [ ]:
list=listdf[0].tolist()
list

In [ ]:

obj = PyPDF2.PdfFileReader(r"PATH/TODOCUMENT.pdf")
pgno = obj.getNumPages()

In [ ]:
#add the difference in value (pagedifference), here. Also consider pages might start numbering later in the pdf (pagecountstart).
#check the original pdf and note these values (for later)
pagedifference=1
pagecountstart=20
#pagecountend=400
#also filter bibliography pages

#check original pdf and manually list bibliography pages
bibliographies=[8,9,10,11,21,22,31,38,44,45,53,54,64,65,73,74,84,92,93,103,115,125,126, 134, 142, 143, 151, 161, 162, 175, 183, 194, 195, 196, 203, 204, 215, 216, 224, 225, 235, 236, 245, 246, 247, 254, 255, 269, 270, 280, 298, 299, 300, 301, 308, 317, 318, 328, 338, 348]

In [ ]:
indexss=[]

for i in range(0, pgno):
    for s in list:
        PgOb = obj.getPage(i)
        Text = PgOb.extractText()
        if re.search(s,Text, re.IGNORECASE):
            indexss.append(str(s)+' === is on page /// '+ str(i))
            print(len(indexss))
            print("String " + str(s) + " Found on Page: " + str(i))

#it takes a while 85 minutes for 400 pages of pdf text and 650 words to index  

In [ ]:
#check that it worked -it is a list 
indexss

Turn the list into dataframe (split string to columns)

In [ ]:
indexdf = pd.DataFrame (indexss, columns = ['All'], index=[0])
indexdf

In [ ]:
indexdf = indexdf.drop_duplicates()

In [13]:
indexdf=indexdf['All'].str.split('///', expand=True)


In [14]:
#make a dirty safety copy to clean
indexdf.to_csv(folder+"indexdirty.csv", encoding='utf-8')

In [15]:
indexdf=indexdf.rename(columns={0:"wordAnd more",1: "pagenumber"}, errors="raise")

In [17]:
indexdf[['word', 'rest']]=indexdf['wordAnd more'].str.split("===", expand=True)

In [38]:
indexdf['row_num'] = indexdf.reset_index().index

In [19]:
indexdf=indexdf.drop(['wordAnd more', 'rest'], axis=1)

In [ ]:
#check the shape of the dataframe
indexdf

In [ ]:
#add the difference in value, here: (in this case it was +1). 
# Also consider pages below 20 are numbered differently.
#remember values before pagecountstart should not be included 

indexdf['pagenumber']=indexdf['pagenumber'].astype('int')
indexdf['pagenumber']=indexdf['pagenumber']+int(pagedifference)
indexdf['pagenumber']=indexdf['pagenumber']-int(pagecountstart)
indexdf.loc[(indexdf.pagenumber < 0),'pagenumber']=-1
for b in bibliographies:
    indexdf.loc[(indexdf.pagenumber == b),'pagenumber']=-100
indexdf

In [ ]:

indexdf.pagenumber.astype('str')
indexdf.loc[(indexdf.pagenumber == -1), 'pagenumber']='prologue'
indexdf.loc[(indexdf.pagenumber == -100),'pagenumber']='bibliography'


Arrange the dataframe

In [69]:
indexdf=(indexdf.groupby(['word'])
      .agg({'pagenumber': lambda x: x.tolist()})
      .reset_index())
indexdf

In [ ]:
#two options: either making lists into str. and cleaning characters... it works better if you have integers and text in the pagenumbers   
indexdf['pagenumber']=indexdf.pagenumber.astype('str')
indexdf['pagenumber'] = indexdf['pagenumber'].str[:-1]
indexdf['pagenumber'] = indexdf['pagenumber'].str[1:]
indexdf

In [80]:
#...or joining the lists into strings (it works better if all page numbers are integers) 
indexdf['pages']=indexdf.pagenumber.apply(lambda x: ', '.join(x))
indexfinal=indexdf.drop(['pagenumber'], axis=1)
indexfinal

In [ ]:
#save final
#for first option
indexdf.to_csv(folder+cleanindex1.csv, encoding='utf-8')
#for second option
indexfinal.to_csv(folder+'cleanindex2.csv', encoding='utf-8')